# Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time

ModuleNotFoundError: No module named 'sklearn'

# Read and Inspect Data

In [ ]:
df = pd.read_csv('Flight_Price_Dataset_Q2.csv')
df

In [ ]:
df['stops'].value_counts()

In [ ]:
df['departure_time'].value_counts()

In [ ]:
df['arrival_time'].value_counts()

In [ ]:
df['class'].value_counts()

# One-Hot Encoding

### Class

In [ ]:
df = pd.get_dummies(df, columns=['class'], drop_first=True)
df

# Label Encoding

### Arrival_time and Departure_time

In [ ]:
mapping = {'Early_Morning': 0,
           'Morning' : 1,
           'Afternoon' : 2,
           'Evening' : 3,
           'Night': 4,
           'Late_Night': 5}

df['arrival_time_encode'] = df['arrival_time'].map(mapping)
df['departure_time_encode'] = df['departure_time'].map(mapping)
df = df.drop('arrival_time', axis = 1)
df = df.drop('departure_time', axis = 1)

df

### Stops

In [ ]:
mapping = {'zero': 0,
           'one' : 1,
           'two_or_more' : 2}

df['stops_encode'] = df['stops'].map(mapping)
df = df.drop('stops', axis = 1)
df

# Feature Correlation 

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(), annot = True, cmap = 'coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Define X and y Sets

In [ ]:
X_df = df.drop(columns=['price'])
y_df = df['price']

In [ ]:
# convert True-False to 1 and 0
X_df['class_Economy'] = X_df['class_Economy'].astype(int)
X_df

# Split Data to Train and Test Sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, shuffle=True)

In [ ]:
X = x_train.to_numpy()
y = y_train.to_numpy()
y = y.reshape(-1, 1)

# Data Normalization

In [ ]:
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
x_test = (x_test - np.mean(x_test, axis=0)) / np.std(x_test, axis=0)

# Prediction and Cost Function

In [ ]:
def predict(W, X, b):
    return X @ W + b

$$f_{w,b}(X) = X^TW  + b$$

In [ ]:
def cost_function(y_hat, y):
    return np.mean((y_hat - y)**2) / 2

$$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2$$ 


# Model Definition

In [ ]:
def gradient_descent(X, y, W, b, alpha):
    n = X.shape[1]
    m = X.shape[0]
    
    error = predict(W, X, b) - y
    dW = X.T @ error / X.shape[0]
    db = np.mean(error)
    
    W -= alpha * dW
    b -= alpha * db
    
    return W, b, error



$$
\frac{\partial J(w,b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})
$$
$$
\frac{\partial J(w,b)}{\partial w}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) -y^{(i)})x^{(i)} 
$$


$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & \phantom {0000} b := b -  \alpha \frac{\partial J(w,b)}{\partial b} \newline       \; & \phantom {0000} w := w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{1}  \; & 
\newline & \rbrace\end{align*}$$

In [ ]:
def model(X, y, max_itr = 1000, convergence_threshold = 0.0001):
    start = time.time()
    
    n = X.shape[1]
    W = np.random.rand(n, 1)
    b = 0
    alpha = 0.01
    cost_history = np.zeros(max_itr)
    itr = 0
    
    for i in range(max_itr):
        W, b, error = gradient_descent(X, y, W, b, alpha)
        cost_history[i] = np.mean(error**2)
        itr += 1
        
        if (i + 1) % 20 == 0:
            print(f"Cost Function on Iteration {i + 1} = {cost_history[i]}")
            
        if i > 0 and cost_history[i - 1] - cost_history[i] < convergence_threshold:
            break
            
    end = time.time()

    return W, b, cost_history, itr, end - start

# Use Model

In [ ]:
W, b, cost_history, itr, ex_time = model(X, y, 400)

# Metrics

In [ ]:
y_pred = predict(W, x_test, b)

In [ ]:
print('R^2:',metrics.r2_score(y_test, y_pred))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
y_pred = predict(W, X, b)

In [ ]:
print('R^2:',metrics.r2_score(y, y_pred))
print('MAE:',metrics.mean_absolute_error(y, y_pred))
print('MSE:',metrics.mean_squared_error(y, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y, y_pred)))

# Plot

In [ ]:
plt.plot(range(1, len(cost_history) + 1), cost_history, color='purple')
plt.xlabel('Number of iterations')
plt.ylabel('Cost J')
plt.title('Gradient Descent: Cost over Iterations')
plt.show()

# Result

In [ ]:
print("PRICE ="
      f"\n{W[0][0]} * {X_df.columns[0]} +", 
      f"\n{W[1][0]} * {X_df.columns[1]} +",
      f"\n{W[2][0]} * {X_df.columns[2]} +",
      f"\n{W[3][0]} * {X_df.columns[3]} +",
      f"\n{W[4][0]} * {X_df.columns[4]} +",
      f"\n{W[5][0]} * {X_df.columns[5]} +",
      f"\n{b}")

In [ ]:
print(f"Training Time: = {ex_time}s")

In [ ]:
print("Logs", f"MSE = {metrics.mean_squared_error(y, y_pred)}",
              f"RMSE = {np.sqrt(metrics.mean_squared_error(y, y_pred))}",
              f"MAE = {metrics.mean_absolute_error(y, y_pred)}",
              f"R2 = {metrics.r2_score(y, y_pred)}", sep = "\n")